In [2]:
import fitz
import io
import cv2
from pyzbar.pyzbar import decode
import numpy
import pandas as pd
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pdfplumber
import os
import glob
import datetime
import re

In [44]:
def leer_qr(nombre_de_archivo):
    with fitz.open(nombre_de_archivo) as pdf:
        page = pdf.load_page(0)
        pix = page.get_pixmap(dpi=300)
        
        pix.save(f"prueba/{nombre_de_archivo}.jpg", "jpg")
    
    # Extract text from pdf
    pdf = pdfplumber.open(nombre_de_archivo)
    page_pdf = pdf.pages[0]
    text = page_pdf.extract_text()
    
    
    nombre = re.findall('Razón Social: (.+?) Fecha de Emisión', text)[0]
    print(f"nombre: {nombre}")
        
    punto_de_venta = re.findall('Punto de Venta: (.+?) Comp. Nro:', text)[0]
    print(f"Punto de venta: {punto_de_venta}")
    comprobante_numero = re.findall('Comp. Nro: (.+?)\n', text)[0]
    print(f"Comprobante numero: {comprobante_numero}")
    cuit = re.findall('CUIT: (.+?) Apellido y Nombre /', text)[0]
    print(f"Cuit: {cuit}")
    cae = re.findall('CAE N°: (.+?)\n', text)[0]
    print(f"Cae: {cae}")
    
    # read qr code
    try:
        img = cv2.imread(f"./prueba/{nombre_de_archivo}.jpg")
        detect = cv2.QRCodeDetector()
        value, points, straight_qrcode = detect.detectAndDecode(img)
        print(f"este es el enlace CAE: {value}")
    except:
        print("No se pudo leer el código QR")

leer_qr("factura 33.pdf")

nombre: PONCE CHRISTIAN NICOLAS
Punto de venta: 00001
Comprobante numero: 00000029
Cuit: 30709165905
Cae: 73099816071751
este es el enlace CAE: 


In [7]:

def read_qr_code(filename, nombre_carpeta):
    """Read an image and read the QR code.
    
    Args:
        filename (string): Path to file
    
    Returns:
        qr (string): Value from QR code
    """
    global text
    global value
    
    # creamos carpeta para guardar imagenes
    fecha = datetime.datetime.now()
    carpeta_imagenes = f"./{fecha.strftime('%Y-%m-%d')}_{nombre_carpeta}"
    print(carpeta_imagenes)
    try:
        os.mkdir(carpeta_imagenes)
        
    except:
        pass

    nombre_de_archivo = os.path.splitext(filename)[0]
    print(nombre_de_archivo)
    global text
    global value
    with fitz.open(filename) as pdf:
        page = pdf.load_page(0)
        pix = page.get_pixmap(dpi=300, alpha=False)
        pix.save(f"{carpeta_imagenes}/{nombre_de_archivo}.jpg", "jpg")
    
    # Extract text from pdf
    pdf = pdfplumber.open(filename)
    page_pdf = pdf.pages[0]
    text = page_pdf.extract_text()
    
   

    # read qr code
    try:
        img = cv2.imread(f"{carpeta_imagenes}/{nombre_de_archivo}.jpg")
        detect = cv2.QRCodeDetector()
        value, points, straight_qrcode = detect.detectAndDecode(img)
        
    except:
        print("No se pudo leer el código QR")
    


TypeError: read_qr_code() missing 1 required positional argument: 'nombre_carpeta'

In [6]:
# Read all pdf files in folder
carpeta = os.path.basename(os.getcwd())

facturas_dict = {}
for file in os.listdir():
    if file.endswith(".pdf"):
        read_qr_code(file, carpeta)
        
        nombre = re.findall('Razón Social: (.+?) Fecha de Emisión', text)[0]
        
            
        punto_de_venta = re.findall('Punto de Venta: (.+?) Comp. Nro:', text)[0]
        
        comprobante_numero = re.findall('Comp. Nro: (.+?)\n', text)[0]
        
        cuit = re.findall('CUIT: (.+?) Apellido y Nombre /', text)[0]
        
        cae = re.findall('CAE N°: (.+?)\n', text)[0]
        

        facturas_dict[file] = {"nombre": nombre, "punto_de_venta": punto_de_venta, "comprobante_numero": comprobante_numero, "cuit": cuit, "cae": cae, "qr": value}


./2023-03-03_3-3
factura 1
./2023-03-03_3-3
factura 10
./2023-03-03_3-3
factura 11
./2023-03-03_3-3
factura 12
./2023-03-03_3-3
factura 13
./2023-03-03_3-3
factura 14
./2023-03-03_3-3
factura 15
./2023-03-03_3-3
factura 16
./2023-03-03_3-3
factura 17
./2023-03-03_3-3
factura 18
./2023-03-03_3-3
factura 19
./2023-03-03_3-3
factura 2


KeyboardInterrupt: 

In [ ]:
nombre = text.split('\n')[3]
punto_de_venta = text.split('\n')[5].split('Comp. Nro: ')[0].split('Punto de Venta: ')[1]
comprobante_numero = text.split('\n')[5].split('Comp. Nro: ')[1]
cuit = text.split('\n')[7].split('CUIT: ')[1]
cae = text.split('\n')[19].split('N°: ')[1]

print(nombre)
print(punto_de_venta)
print(comprobante_numero)
print(cuit)
print(cae)
print(value)

In [52]:
#diccionario to csv

df = pd.DataFrame.from_dict(facturas_dict, orient='index')
df.head()



,nombre,punto_de_venta,comprobante_numero,cuit,cae,qr
factura 1.pdf,LONGHI HUGO FERNANDO,00003,00000012,30709165905,73090931346909,https://www.afip.gob.ar/fe/qr/?p=eyJ2ZXIiOjEsI...
factura 10.pdf,HERNANDEZ RODRIGO,00002,00000032,30709165905,73089324316110,https://www.afip.gob.ar/fe/qr/?p=eyJ2ZXIiOjEsI...
factura 11.pdf,GIMENEZ FLORENCIA ALEJANDRA,00001,00000041,30709165905,73089318812827,https://www.afip.gob.ar/fe/qr/?p=eyJ2ZXIiOjEsI...
factura 12.pdf,FERNANDEZ LORENA ISAURA,00001,00000004,30709165905,73081316034598,https://www.afip.gob.ar/fe/qr/?p=eyJ2ZXIiOjEsI...
factura 13.pdf,ARDUSSO SOFIA,00003,00000028,30709165905,73082410286846,https://www.afip.gob.ar/fe/qr/?p=eyJ2ZXIiOjEsI...


In [54]:
fecha = datetime.datetime.now()
carpeta_excel = f"./{fecha.strftime('%Y-%m-%d')}_Excel_con_datos"
os.mkdir(carpeta_excel)
df.to_excel(f"{carpeta_excel}/facturas.xlsx")